In [2]:
c(1,2,3) + c(1,2,3)

[1] 2 4 6

# SETUP

In [13]:
source('Classes.R')

In [14]:
sum_of_squares<-function(point){
   sum <- 0.0        
   for(p in point)
   {
     sum <- sum + p^2
   }
   return(sum)
}

gen_random_population <- function(size=lambda, scaler=1){
    coordinates <- scaler * replicate(size, runif(n = point_dim, min = -1, max = 1))
    fitness <- c()
    for(i in 1:dim(coordinates)[2]){
        fitness[i]<-objective_fun$evaluate(coordinates[,i])
    }
    population <- data.frame(t(coordinates), fitness)
    return(population)
}

In [15]:
# ARGUMENTS
point_dim <- 3
lambda <- 50
fun <- sum_of_squares 

In [16]:
objective_fun <- ObjectiveFunction$new(fun=fun, dim=point_dim)

# DES

In [17]:
library("ringbuffer")
N <- point_dim

In [18]:
# PARAMETER INIT
Ft          <- 1                                ## Scaling factor of difference vectors (a variable!)                   ## Fitness value after which the convergence is reached
## Strategy parameter setting:                                          ## Population size
mu          <- floor(lambda/2)   

weights     <- log(mu+1) - log(1:mu)       
weights     <- weights/sum(weights)                                 ##    \-> weights are normalized by the sum
weightsSumS <- sum(weights^2)
weightsPop      <- log(lambda+1) - log(1:lambda)
weightsPop      <- weightsPop/sum(weightsPop)

# parameters that can be set manually
mueff       <- sum(weights)^2/sum(weights^2) ## Variance effectiveness factor
cc          <- mu/(mu+2)                      ## Evolution Path decay factor
pathLength  <- 6                       ## Size of evolution path
cp          <- 1/sqrt(N)                        ## Evolution Path decay factor      ## Maximum number of iterations after which algorithm stops
c_Ft        <- 0
pathRatio   <- sqrt(pathLength)           ## Path Length Control reference value
histSize    <- ceiling(6+ceiling(3*sqrt(N)))## Size of the window of history - the step length history
Ft_scale    <- ((mueff+2)/(N+mueff+3))/(1 + 2*max(0, sqrt((mueff-1)/(N+1))-1) + (mueff+2)/(N+mueff+3))
tol         <- 10^-20
counteval   <- 0                                                    ## Number of function evaluations
sqrt_N      <- sqrt(N)

# best.fit        <- Inf                  ## The best fitness found so far
# best.par        <- NULL                 ## The best solution found so far
# worst.fit       <- NULL                 ## The worst solution found so far:
# last.restart    <-0
# restart.length  <-0
# restart.number  <-0

# dynamic parameter init
steps       <- ringbuffer(size = pathLength*N)                      ## Cyclical buffer containing last 'pathLength' steps of algorithm
dMean       <- array(0, dim=c(N,histSize))
FtHistory   <- array(0, histSize)                                   ## Array buffer containing 'histSize' last values of 'Ft'
pc       <- array(0, dim=c(N,histSize))

# history init
histHead    <- 0                                                      ## Pointer to the history buffer head
iter        <- 0L                                                     ## Number of iterations
history     <- list() 

In [19]:
population_2_matrix <- function(df){
    return(t(data.matrix(df[,!(names(df)=='fitness')])))
}

matrix_2_population <- function(mat){
    fitness <- c()
    for(i in 1:dim(mat)[2]){
        fitness[i]<-objective_fun$evaluate(mat[,i])
    }
    return(data.frame(t(mat), fitness))
}

selection <- function(df, offspring_num){
    df_sorted <- df[order(df['fitness'], decreasing=FALSE),]
    selected_points <- df_sorted[1:offspring_num,]
    return(selected_points)
}

sampleFromHistory <- function(history,historySample,lambda){
  ret <- c()
  for(i in 1:lambda)
    ret <- c(ret,sample(1:ncol(history[[historySample[i]]]), 1))
  return(ret)
}

In [20]:
population_df <- gen_random_population()
population_mat <- population_2_matrix(population_df)

# cumMean=(upper+lower)/2
# populationRepaired <- apply(population,2,bounceBackBoundary2)

# if(Lamarckism==TRUE){
#   population <- populationRepaired
# }

# selection       <- rep(0, mu)
# selectedPoints  <- matrix(0, nrow=N, ncol=mu)
# fitness         <- fn_l(population)
oldMean         <- numeric(N)
newMean         <- c(10,10,10)
# limit           <- 0
worst.fit       <- max(population_df$fitness)

# Store population and selection means
popMean         <- drop(population_mat %*% weightsPop)
muMean          <- newMean

## Matrices for creating diffs
diffs     <- matrix(0, N, lambda)
x1sample  <- numeric(lambda)
x2sample  <- numeric(lambda)

chiN      <- (sqrt(2)*gamma((N+1)/2)/gamma(N/2))
histNorm  <- 1/sqrt(2)
counterRepaired <- 0

stoptol=F

In [22]:
for(it in 1:1){
iter      <- iter + 1L
histHead  <- (histHead %% histSize) + 1

selectedPoints_df <- selection(population_df, offspring_num=mu)
selectedPoints_mat <- population_2_matrix(selectedPoints_df)

# Save selected population in the history buffer
history[[histHead]] <- array(0,dim=c(N,mu))
history[[histHead]] <- selectedPoints_mat * histNorm/Ft

## Calculate weighted mean of selected points
oldMean <- newMean
newMean <- drop(selectedPoints_mat %*% weights)

## Write to buffers
muMean <- newMean
dMean[,histHead] <- (muMean - popMean) / Ft

step <- (newMean - oldMean) / Ft

## Update Ft
FtHistory[histHead] = Ft
oldFt <- Ft

if(histHead==1){
pc[,histHead] = (1 - cp)* rep(0.0, N)/sqrt(N) + sqrt(mu * cp * (2-cp))* step
}else{
pc[,histHead] = (1 - cp)* pc[,histHead-1] + sqrt(mu * cp * (2-cp))* step
}
## Sample from history with uniform distribution
limit <- ifelse(iter < histSize, histHead, histSize)
historySample <- sample(1:limit,lambda, T)
historySample2 <- sample(1:limit,lambda, T)

x1sample <- sampleFromHistory(history,historySample,lambda)
x2sample <- sampleFromHistory(history,historySample,lambda)

## Make diffs
for (i in 1:lambda) {
x1 <- history[[historySample[i]]][,x1sample[i]]
x2 <- history[[historySample[i]]][,x2sample[i]]

diffs[,i] <- sqrt(cc)*( (x1 - x2) + rnorm(1)*dMean[,historySample[i]] ) + sqrt(1-cc) * rnorm(1)*pc[,historySample2[i]]

}

## New population
population_mat <- newMean + Ft * diffs + tol*rnorm(diffs)/chiN
# population_mat <- deleteInfsNaNs(population_mat)

population_df <- matrix_2_population(population_mat)

# Check constraints violations
# Repair the individual if necessary
# populationTemp <- population
# # populationRepaired <- apply(population,2,bounceBackBoundary2)

# counterRepaired=0
# for(tt in 1:ncol(populationTemp)){
# if(any(populationTemp[,tt] != populationRepaired[,tt]))
#   counterRepaired = counterRepaired + 1
# }

# if(Lamarckism==TRUE){
# population <- populationRepaired
# }

popMean <- drop(population_mat %*% weightsPop)

## Evaluation
# fitness <- fn_l(population)
# if(Lamarckism==FALSE){
# fitnessNonLamarcian <- fn_d(population, populationRepaired, fitness)
# }

## Break if fit :
# wb <- which.min(fitness)
# if (fitness[wb] < best.fit) {
# best.fit <- fitness[wb]
# if(Lamarckism==TRUE)
#   best.par <- population[,wb]
# else
#   best.par <- populationRepaired[,wb]
# }

## Check worst fit:
# ww <- which.max(fitness)
# if (fitness[ww] > worst.fit){
# worst.fit <- fitness[ww]
# }

## Fitness with penalty for nonLamarcian approach
# if(Lamarckism==FALSE){
# fitness <- fitnessNonLamarcian
# }


## Check if the middle point is the best found so far
# cumMean <- 0.8*cumMean+0.2*newMean
# cumMeanRepaired <-bounceBackBoundary2(cumMean)

# fn_cum  <- fn_l(cumMeanRepaired)
# if (fn_cum < best.fit) {
# best.fit <- drop(fn_cum)
# best.par <- cumMeanRepaired
# }

## Escape from flat-land:
#if (min(fitness) == sort(fitness,partial=min(1+floor(lambda/2), 2+ceiling(lambda/4)))[min(1+floor(lambda/2), 2+ceiling(lambda/4))]) {
#  Ft <- Ft * exp(0.2*Ft_scale);
#}

if (population_df$fitness[1] <= 10e-30) {
msg <- "Stop fitness reached."
break
}

# if(abs(range(fitness)[2] - range(fitness)[1]) < tol)
# {
# if (counteval < 0.8*budget)
#   stoptol=T
# }
}

In [13]:
population_df[order(population_df$fitness),]$fitness[1]

[1] 2.495124e-25

In [60]:
diffs

-9.117790,-4.770637,9.680561,-11.05864,1.519108,7.124435,1.531438,-8.681174,6.412828,-24.88766,...,-3.840824,-11.36312,-19.42965,8.777264,-15.40136,6.223980,7.928004,24.17155,3.032713,-2.370080
-8.607163,-4.797802,10.481526,-11.87481,1.627184,6.428388,1.168930,-8.570410,5.917455,-24.99291,...,-3.190307,-12.30551,-19.23578,8.466165,-15.15131,6.264006,7.228293,24.41149,2.797984,-3.014889
-8.945059,-3.913683,10.003018,-12.41196,1.166104,6.620429,1.291629,-8.367997,5.992184,-24.67590,...,-4.530598,-11.28952,-18.79929,8.547980,-15.76884,7.441960,8.174620,24.20776,2.505923,-2.000142


In [68]:
rnorm(diffs)

[1]  0.1702905934 -1.3184277135  0.2542041476 -0.0509344288  0.3477285881
  [6] -0.6237993328 -0.7431107748  0.5479000412  0.2976255315  0.6734191920
 [11]  0.8502850632 -0.5768513899 -1.9922094191 -0.8647585665 -1.5995348618
 [16]  0.9188624400 -0.4847645012 -0.4945186900  0.5177058405  0.9246015423
 [21]  0.3367814594  1.8690792439 -0.9945382982 -0.8886116593 -0.6736039470
 [26]  0.5311138784  0.1916439462  0.4273093045 -0.4040728885 -1.1270953185
 [31] -0.1118521373 -1.1113428360  0.5027009491  1.5187610431 -0.0433529412
 [36]  0.1349345079  0.3978790088 -0.4345281161  0.4405840191  0.6952448853
 [41]  0.9085216901 -0.7293994182  1.0594300325  0.7239654634  0.8300442575
 [46]  1.8993593278  0.0082964258  0.7224708019 -0.2813350481 -0.1733143286
 [51]  1.4699653788  1.0676142567 -1.7413575409  0.0009866389 -0.7560942418
 [56] -1.1401532580 -0.1870737211  1.0887600998 -0.5207031048  1.8692908923
 [61]  0.9608501601 -0.6682445799  0.4935700672  0.0261371919 -0.0142409766
 [66]  0.5501503713  0.7133510938  1.6873670071 -0.5478404986 -2.9933286954
 [71]  0.5004836929 -0.0674946869  1.7868504289  0.9517449198  0.9132538679
 [76]  0.9160574009 -0.0630320481 -3.2621346122  0.8935418239 -1.6819189909
 [81]  1.4964999651  1.8367808960 -0.7036933462 -0.1236896097 -0.7098648800
 [86] -0.0822498377  0.1372585390  0.8973833434 -1.4079697913 -0.0529636131
 [91]  0.3906170954 -0.3102318578 -1.2218932884 -0.8794593577 -0.0881680945
 [96] -1.3266817269  0.1544785620 -0.2469719722 -1.0206366681 -1.4139251749
[101] -0.4442937345 -0.1406302728 -0.3960929231  0.1501201047  0.1907774378
[106] -0.0328282449  0.3895616829  1.6032517739  0.0850842966 -0.6364752351
[111]  0.1154002087 -0.2822712415 -0.0104090173 -0.2571283394  1.1451350519
[116] -0.1727056939  0.9148251366 -0.0779248989  0.9773244713  0.4723969908
[121] -0.2718009246  0.8873768265 -0.1903109897 -0.7457054229 -0.2569664005
[126] -1.4868296281 -2.0264394367 -1.4253483115 -0.0828398445  1.8502247026
[131] -1.5043696336  0.4996698633 -0.2560339889 -1.7708582287  0.0635607586
[136] -0.0443475233  1.0302888669 -0.3655806333  1.5374870086  0.9789325361
[141] -0.5124703346  1.5630303513 -0.4354696711 -0.4743865176  0.5696452335
[146]  0.9902514021 -1.0339682453 -0.2625074713  1.6145509434  0.9785808534

In [69]:
newMean + Ft * diffs + tol*rnorm(diffs)/chiN

-9.100881,-4.753728,9.697470,-11.04173,1.536018,7.141344,1.548347,-8.664265,6.429737,-24.87075,...,-3.823915,-11.34621,-19.41274,8.794173,-15.38446,6.240889,7.944913,24.18846,3.049622,-2.353171
-8.652594,-4.843233,10.436095,-11.92024,1.581753,6.382957,1.123499,-8.615841,5.872023,-25.03835,...,-3.235738,-12.35094,-19.28121,8.420733,-15.19674,6.218575,7.182862,24.36606,2.752553,-3.060321
-8.955525,-3.924149,9.992551,-12.42242,1.155638,6.609962,1.281163,-8.378463,5.981718,-24.68637,...,-4.541064,-11.29999,-18.80976,8.537514,-15.77931,7.431494,8.164154,24.19729,2.495457,-2.010608


In [73]:
tol*rnorm(diffs)

[1] -1.312123e-20 -1.903284e-21 -1.772139e-21  1.027481e-20 -1.960384e-21
  [6] -1.057228e-21  1.456659e-20 -6.217599e-21 -1.377521e-20  1.968017e-21
 [11]  2.305102e-21 -5.740120e-21 -7.555018e-21  2.501377e-21  1.981061e-20
 [16]  1.276888e-20  5.502975e-21 -6.457464e-21  5.265770e-21  2.799918e-21
 [21]  3.411824e-21  1.001306e-21  5.959069e-21  1.098765e-20  3.030469e-21
 [26]  1.636360e-20  4.455494e-21 -1.528212e-22 -9.207699e-21 -4.998377e-21
 [31] -4.279634e-21  9.609043e-21  8.186997e-21  5.017270e-21  4.115348e-21
 [36] -1.224031e-20 -5.011362e-21 -1.132003e-20 -1.264831e-20  1.004643e-20
 [41] -3.066823e-21 -1.540236e-20  9.949820e-21  2.064361e-20  1.149728e-20
 [46]  2.082381e-22 -6.904869e-21  7.093811e-21  1.896367e-21  1.325406e-20
 [51] -3.918632e-21 -8.370663e-21 -4.609897e-21  1.653892e-21 -1.933943e-20
 [56]  1.692368e-20  1.189073e-22  1.205943e-20  2.059072e-21  4.642898e-21
 [61] -1.614632e-20 -6.462047e-21 -2.473696e-21 -2.756259e-21 -3.847702e-22
 [66]  1.362325e-21  1.655297e-21  7.298935e-21  6.990405e-22 -6.104741e-21
 [71]  9.534004e-21  7.917663e-21  9.445328e-21  5.880305e-21 -1.297674e-20
 [76]  9.501290e-21 -2.155289e-21  1.203867e-21  5.781118e-21  8.781406e-21
 [81] -1.997936e-20 -1.146489e-20 -8.007683e-21 -1.450699e-21  1.601880e-21
 [86]  1.480441e-20  3.495363e-20 -1.182356e-21  4.457240e-21 -1.931965e-20
 [91] -1.078334e-20  1.593723e-21  4.999494e-21 -3.304250e-21 -7.499226e-21
 [96]  7.108654e-21 -1.011757e-20 -7.912105e-23  1.278002e-20 -6.554677e-21
[101]  7.179926e-21  4.003204e-21 -2.460645e-20 -1.766675e-20  8.324463e-21
[106] -3.128216e-22 -3.380851e-21  4.734277e-21  7.172780e-21 -1.355419e-21
[111]  8.420193e-22  2.015112e-21 -1.461982e-21 -9.628617e-21  1.946950e-20
[116]  7.213909e-21 -2.622683e-21  1.587449e-20  9.832178e-22 -1.040482e-20
[121] -1.233758e-20  1.059070e-20  1.193753e-20 -8.688360e-21 -1.165079e-20
[126]  9.784765e-21  1.663694e-21  6.024795e-21  4.258181e-21  1.529405e-20
[131]  1.333502e-21 -2.269456e-21 -9.414686e-21 -4.650908e-21 -2.089617e-21
[136] -1.159393e-20  1.411204e-20 -9.880486e-21  1.523428e-21 -6.442603e-21
[141] -8.128093e-21  7.722210e-21  2.063108e-20  1.395721e-20 -3.389115e-21
[146]  1.052146e-21 -4.622991e-21  3.573814e-21  1.486590e-20 -1.113038e-21

In [37]:
sample(1:10, 50, T)

[1]  9  9  2  9  9  3  4 10  7  9  1  4  1  2  1  5  2  1  7  3  7  1  3  4  2
[26]  1  6  7  3  7  5  9  3  8  9  9  8  1  9  3 10  8  4  7  5  2  1  4  8  6

In [38]:
history

,7,6,11,13,30,17,19,20,9,10,...,44,48,39,34,28,47,35,16,26,12
X1,0.1349544,0.1213178,-0.20989371,0.37558995,-0.2013951,0.0807394,-0.24478575,-0.1517751,0.2490053,-0.2981597,...,-0.54636434,0.46705817,0.01882672,0.02780434,0.05075018,-0.48245558,-0.3156803,0.34621334,-0.1159303,0.1091973
X2,0.0847133,0.3088177,0.30924255,0.06154902,-0.3313912,-0.3670462,-0.05058477,-0.3723465,-0.3371132,-0.1875974,...,-0.04037370,0.07775235,0.18304233,0.52805285,0.58712974,0.36782579,-0.1686738,-0.08429798,-0.5476030,0.3168271
X3,0.1830761,-0.1429918,0.05260071,0.02881306,0.1112896,-0.1502844,0.34862227,0.1750779,-0.1686331,-0.3724721,...,-0.09941048,-0.29561644,-0.54975982,-0.25995361,0.06271807,0.05490435,0.4994616,0.51056499,0.2802775,-0.5511214


In [39]:
historySample <- sample(1:limit,lambda, T)
historySample2 <- sample(1:limit,lambda, T)

x1sample <- sampleFromHistory(history,historySample,lambda)

In [45]:
historySample

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[39] 1 1 1 1 1 1 1 1 1 1 1 1

In [46]:
sampleFromHistory <- function(history,historySample,lambda){
  ret <- c()
  for(i in 1:lambda)
    ret <- c(ret,sample(1:ncol(history[[historySample[i]]]), 1))
  return(ret)
}

In [47]:
history[[historySample[1]]]

,7,6,11,13,30,17,19,20,9,10,...,44,48,39,34,28,47,35,16,26,12
X1,0.1349544,0.1213178,-0.20989371,0.37558995,-0.2013951,0.0807394,-0.24478575,-0.1517751,0.2490053,-0.2981597,...,-0.54636434,0.46705817,0.01882672,0.02780434,0.05075018,-0.48245558,-0.3156803,0.34621334,-0.1159303,0.1091973
X2,0.0847133,0.3088177,0.30924255,0.06154902,-0.3313912,-0.3670462,-0.05058477,-0.3723465,-0.3371132,-0.1875974,...,-0.04037370,0.07775235,0.18304233,0.52805285,0.58712974,0.36782579,-0.1686738,-0.08429798,-0.5476030,0.3168271
X3,0.1830761,-0.1429918,0.05260071,0.02881306,0.1112896,-0.1502844,0.34862227,0.1750779,-0.1686331,-0.3724721,...,-0.09941048,-0.29561644,-0.54975982,-0.25995361,0.06271807,0.05490435,0.4994616,0.51056499,0.2802775,-0.5511214


In [49]:
ncol(history[[historySample[i]]])

[1] 25

In [56]:
rnorm()

ERROR: Error in rnorm(): brakuje argumentu 'n', a nie ma określonej wartości domyślnej
